In [2]:
# Import the modules
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
import sqlite3


## 1. Split the Data into Training and Testing Sets

### 1.1 Read the preproccessed Pokemon data from the `Resources` folder into a Pandas DataFrame.

In [ ]:
# Read the sqlite file from the Resources folder into a Pandas DataFrame
cnx = sqlite3.connect('<file name>.db')

# Review the DataFrame
df = pd.read_sql_query("SELECT * FROM table_name", cnx)
df

### 1.2 Create Target and Features
- Create the target (`y`)  from the “Did_the_first_pokemon_win?” column
- create the features (`X`) DataFrame from the remaining columns.

In [ ]:
# Separate the data into target and features

# Separate the y variable, the target
y = df['Did_the_first_pokemon_win?']

# Separate the X variable, the features
X = df.copy()
X.drop('Did_the_first_pokemon_win?', axis=1, inplace=True)

In [ ]:
# Review the X variable DataFrame
X.head()

### 1.3 Check the balance of the target variable (`y`) by using the `value_counts` function.

In [ ]:
# Check the balance of our target values
y.value_counts()

### 1.4 Split the data into training and testing datasets by using `train_test_split`.


In [ ]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
#adding 'stratify' to ensure that the random sample matches the demos of the total sample

#CHRISTIN - CHECK THAT STRATIFY IS NEEDED HERE WITH VALUE COUNTS

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)

## 2. Create a Logistic Regression Model with the Original Data

###  2.1 Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

In [ ]:
#Validate the model w/ training data
#Use method 'score' to return the mean accuracy on the given test data and targets.

classifier.score(X_train, y_train)

In [ ]:
#Validate the model w/ testing data
#Use method 'score' to return the mean accuracy on the given test data and targets.
classifier.score(X_test, y_test)

### 2.2 Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
#code

### 2.3 valuate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
#code

## 3. Save and Reload the Model using Pickle

In [ ]:
#Reference
#https://www.geeksforgeeks.org/saving-a-machine-learning-model/

#save the model with pickle
#pickle.dump serializes an object hierarchy

#CHRISTIN define model above
saved_model = pickle.dumps(model, open('model.pkl', 'wb'))

In [ ]:
#load the pickle model
pickled_model = pickle.loads(open('model.pkl', 'rb'))

#use the loaded model to make predictions
pickled_model.predict(X_test)
